In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.6.0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense,Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:
import zipfile

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/dataset/CovidDataset.zip', 'r')
zip_ref.extractall('/content/drive/MyDrive/dataset/Covid') 
zip_ref.close()

In [ ]:
os.chdir("/content/drive/MyDrive/dataset/Covid/CovidDataset/Train/Covid")
!ls

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Covid/CovidDataset/Train',
                                                 target_size = (180, 180),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 5800 images belonging to 2 classes.


In [ ]:
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Covid/CovidDataset/Val',
                                            target_size = (180, 180),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1305 images belonging to 2 classes.


In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[180, 180, 3]))
  model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
  model.add(Dropout(rate=0.20))

  model.add(tf.keras.layers.Flatten())

  model.add(tf.keras.layers.Dense(units=128, activation='relu'))
  model.add(Dropout(rate = 0.25))
  model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [ ]:
# Create a basic model instance
model = create_model()

In [ ]:
# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 43, 43, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 118336)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [ ]:
checkpoint_path = "/content/drive/MyDrive/saved/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(x = training_set, validation_data = test_set, epochs = 30,
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [ ]:
checkpoint_path = "/content/drive/MyDrive/saved/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [ ]:
# Part 4 - Making a single prediction
print(training_set.class_indices)

{'Covid': 0, 'Normal': 1}


In [ ]:
test_image = image.load_img('/content/drive/MyDrive/dataset/Covid/CovidDataset/single_prediction/Normal-3557.png', target_size = (180, 180))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

if result[0][0] == 1:
    prediction = 'Normal'
else:
    prediction = 'Covid'
print(prediction)

Normal


In [ ]:
test_image = image.load_img('/content/drive/MyDrive/dataset/Covid/CovidDataset/single_prediction/COVID-3614.png', target_size = (180, 180))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

if result[0][0] == 1:
    prediction = 'Normal'
else:
    prediction = 'Covid'
print(prediction)

Normal
